In [1]:
import random

suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 'Nine':9, 'Ten':10, 'Jack':10,
             'Queen':10, 'King':10, 'Ace':11}

playing = True


class Card:
    
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
    
    def __str__(self):
        return self.rank + " of " + self.suit


class Deck:
    
    def __init__(self):
        self.deck = []  # start with an empty list
        for suit in suits:
            for rank in ranks:
                self.deck.append(Card(suit, rank))
    
    def __str__(self):
        deck_comp = ""
        for everycard in self.deck:
            deck_comp += "\n" + everycard.__str__()
        return "The deck has: " + deck_comp

    def shuffle(self):
        random.shuffle(self.deck)
        
    #grab a card from the deck to the Dealer and a card to the Player
    def deal(self):
        single_card = self.deck.pop()
        return single_card


test_deck = Deck()
test_deck.shuffle()
print(test_deck)


class Hand:
    def __init__(self):
        self.cards = []  # start with an empty list as we did in the Deck class
        self.value = 0   # start with zero value
        self.aces = 0    # add an attribute to keep track of aces
    
    #grab cards from the Deck and add to player's hand - from Deck.deal()
    def add_card(self,card):
        self.cards.append(card)
        
        #take card's rank -> look up in the self.values dictionary for it's value -> add to self.value
        self.value += values[card.rank]

        #track aces to adjust value (aces can be 1 or 11 based on player's choice)
        if card.rank == "Ace":
            self.aces += 1
    
    def adjust_for_ace(self):
        #ace were considered to be 11 at beginning. If total value>21 and still have an Ace -> change ace to be a 1 instead of an 11
        while self.value > 21 and self.aces: #aka self.aces > 0
            #adjust value from 11 to 1, deduct 10 from previous value (Ace was 11, now 1)
            self.value -= 10
            #after adjust ace, eliminate the ace used from self.aces track
            self.aces -= 1



class Chips:
    
    def __init__(self, total = 100):
        self.total = total  # This can be set to a default value or supplied by a user input
        self.bet = 0
        
    def win_bet(self):
        self.total += self.bet
    
    def lose_bet(self):
        self.total -= self.bet


def take_bet(chips):
    while True:
        #check if bet is eligible
        try:
            chips.bet = int(input("How many chips would you like to bet? "))
        except:
            #check if player input a number
            print("Sorry please provide an integer")
        else:
            #if it is a number, check if player has enough chips
            if chips.bet > chips.total:
                print(f"Sorry you don not have enough chips. You have {chips.total}")
            else:
                break


def hit(deck,hand):
    #add one card to hand from deck
    hand.add_card(deck.deal())
    #check for ace adjustments
    hand.adjust_for_ace()


def hit_or_stand(deck,hand):
    #assign playing as a global variable
    global playing  # to control an upcoming while loop
    
    while True:
        x = input("hit or stand? ")
        if x == "hit":
            hit(deck,hand)
        elif x == "stand":
            print("player stands, dealer's turn")
            playing = False
        else:
            print("sorry, i did not understand that, please enter 'hit' or 'stand' ")
            continue
        break


def show_some(player,dealer):

    #dealer.cards[1]

    #show 1 of the dealer's hand
    print("\ndealer's hand: ")
    print("<first card hidden> ")
    #show the second card in dealer's hand
    print(dealer.cards[1])
    #show all(2cards) of the player's hand
    print("\nPlayer's hand: ")
    for card in player.cards:
        print(card)

def show_all(player,dealer):
    #show all the dealers's cards
    print("\ndealer's hand: ")
    for card in dealer.cards:
        print(card)
    #calculate and display the value of cards
    print(f"value of dealer's hand is: {dealer.value}")

    #show all the player's cards
    print("\nPlayer's hand: ")
    for card in player.cards:
        print(card)
    print(f"value of player's hand is: {player.value}")


def player_busts(player, dealer, chips):
    print("player busts")
    #player wins bets
    chips.lose_bet()

def player_wins(player, dealer, chips):
    print("player wins")
    chips.win_bet()

def dealer_busts(player, dealer, chips):
    print("dealer busts, player wins")
    chips.win_bet()
    
def dealer_wins(player, dealer, chips):
    print('dealer wins, player lost')
    chips.lose_bet()
    
def push(player, dealer):
    #tie
    print("dealer and player tie! push ")


# Set up the Player's chips
player_chips = Chips()
while True:
    # Print an opening statement
    print("welcome to blackjack")
    
    # Create & shuffle the deck, deal two cards to each player
    deck = Deck()
    deck.shuffle()
    
    #create player's hand of cards, twice (2cards)
    player_hand = Hand()
    player_hand.add_card(deck.deal())
    player_hand.add_card(deck.deal())

    #create dealer's hand of cards
    dealer_hand = Hand()
    dealer_hand.add_card(deck.deal())
    dealer_hand.add_card(deck.deal())


    
    # Prompt the Player for their bet
    take_bet(player_chips)
    
    # Show cards (but keep one dealer card hidden)
    show_some(player_hand, dealer_hand)
    
    while playing:  # recall this variable from our hit_or_stand function
        
        # Prompt for Player to Hit or Stand
        hit_or_stand(deck, player_hand)
        
        # Show cards (but keep one dealer card hidden)
        show_some(player_hand,dealer_hand)
        
        # If player's hand exceeds 21, run player_busts() and break out of loop
        if player_hand.value > 21:
            player_busts(player_hand, dealer_hand, player_chips)
            break
                
    # If Player hasn't busted, play Dealer's hand until Dealer reaches 17
    if player_hand.value <= 21:
        while dealer_hand.value < 17:
            hit(deck, dealer_hand)
    
        # Show all cards
        show_all(player_hand, dealer_hand)
        # Run different winning scenarios
        if dealer_hand.value > 21:
            dealer_busts(player_hand, dealer_hand, player_chips)
        elif dealer_hand.value > player_hand.value:
            dealer_wins(player_hand, dealer_hand, player_chips)
        elif dealer_hand.value < player_hand.value:
            player_wins(player_hand, dealer_hand, player_chips)
        else:
            push(player_hand, dealer_hand)
        
    # Inform Player of their chips total 
    print(f"\nplayer total chips: {player_chips.total}")

    # Ask to play again
    new_game = input("play again? y/n ")
    if new_game == "y":
        playing = True
        continue
    else:
        print("thank you for playing")
        break